# This is a Program to count number of faces in a frame

#### Apart from counting the number of faces in the frame, this program can also Identify the faces of people present given that the model has been trained to identify them. Currently the model can Identify
###### - Lionel Messi
###### - Maria Sharapova
###### - Roger Federer
###### - Serena Williams
###### - Roger Federer
#### You can train more faces if you have the dataset available.

In [1]:
from pygrabber.dshow_graph import FilterGraph
import joblib
import numpy as np
import cv2
import pywt
import matplotlib
import base64
import json
from matplotlib import pyplot as plt
%matplotlib inline


In [2]:
__class_name_to_number = {}
__class_number_to_name = {}

__model = None

In [3]:
def class_number_to_name(class_num):
    return __class_number_to_name[class_num]

In [4]:
def w2d(img, mode='haar', level=1):
    imArray = img
    #Datatype conversions
    #convert to grayscale
    imArray = cv2.cvtColor( imArray,cv2.COLOR_RGB2GRAY )
    #convert to float
    imArray =  np.float32(imArray)
    imArray /= 255
    # compute coefficients
    coeffs=pywt.wavedec2(imArray, mode, level=level)

    #Process Coefficients
    coeffs_H=list(coeffs)
    coeffs_H[0] *= 0

    # reconstruction
    imArray_H=pywt.waverec2(coeffs_H, mode)
    imArray_H *= 255
    imArray_H =  np.uint8(imArray_H)

    return imArray_H

In [5]:
def load_saved_artifacts():
    print("loading saved artifacts...start")
    global __class_name_to_number
    global __class_number_to_name

    with open("./class_dictionary.json", "r") as f:
        __class_name_to_number = json.load(f)
        __class_number_to_name = {v:k for k,v in __class_name_to_number.items()}

    global __model
    if __model is None:
        with open('./saved_model.pkl', 'rb') as f:
            __model = joblib.load(f)
    print("loading saved artifacts...done")

In [6]:
def get_cropped_image_if_2_eyes(img):
    face_cascade = cv2.CascadeClassifier('./opencv/haarcascades/haarcascade_frontalface_default.xml')
    eye_cascade = cv2.CascadeClassifier('./opencv/haarcascades/haarcascade_eye.xml')

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    cropped_faces = []
    for (x,y,w,h) in faces:
            roi_gray = gray[y:y+h, x:x+w]
            roi_color = img[y:y+h, x:x+w]
            eyes = eye_cascade.detectMultiScale(roi_gray)
            if len(eyes) >= 2:
                cropped_faces.append(roi_color)
    return cropped_faces

In [7]:
def classify_image(image):

    imgs = get_cropped_image_if_2_eyes(image)

    result = []
    for img in imgs:
        scalled_raw_img = cv2.resize(img, (32, 32))
        img_har = w2d(img, 'db1', 5)
        scalled_img_har = cv2.resize(img_har, (32, 32))
        combined_img = np.vstack((scalled_raw_img.reshape(32 * 32 * 3, 1), scalled_img_har.reshape(32 * 32, 1)))

        len_image_array = 32*32*3 + 32*32

        final = combined_img.reshape(1,len_image_array).astype(float)
        result.append({
            'class': class_number_to_name(__model.predict(final)[0]),
            'class_probability': np.around(__model.predict_proba(final)*100,2).tolist()[0],
            'class_dictionary': __class_name_to_number
        })
            
    return result

### Load ML Model

In [8]:
if __name__ == '__main__':
    load_saved_artifacts()

loading saved artifacts...start
loading saved artifacts...done


### Get Device list and select device

In [9]:
graph = FilterGraph()
device_list = graph.get_input_devices()
print('Device:\t\t\tIndex')
for i in range(len(device_list)):
    print(device_list[i], '-\t', i)

print("\nSelect index your input device: ")
device_num = int(input())

Device:			Index
USB2.0 HD UVC WebCam -	 0

Select index your input device: 
0


### Start Camera

In [11]:
face_cascade = cv2.CascadeClassifier('./opencv/haarcascades/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('./opencv/haarcascades/haarcascade_eye.xml')

c = cv2.VideoCapture(device_num)

while(1):
    _,feed = c.read()
    face_img = feed.copy()
    gray = cv2.cvtColor(feed, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    people = []
    try:
        for (x,y,w,h) in faces:
            cv2.rectangle(face_img,(x,y),(x+w,y+h),(255,0,0),2)
            res = classify_image(feed)
            for i in res:
                if i not in people:
                    people.append(i['class'])
            try:
                cv2.putText(face_img, f'No. of faces : = {len(faces)}', (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1,
                                (209, 80, 0, 255), 1)
            except:
                pass
        cv2.imshow('Detected face', face_img)
    except Exception:
        pass
    
    cv2.imshow('Capturing Feed',feed)
    
    if cv2.waitKey(5)==27:
        break

c.release()
cv2.destroyAllWindows()
if people:
    '''
    List that can be Identified: 
    - Virat Kohli
    - Roger Federer
    - Maria Sharapova
    - Serena Williams
    - Lionel Messi
    '''
    print("List of people Identified: ")
    print(people)
else:
    print(None)

None
